# Playground 1: Predicting titanic survival using XGBoost

We will use the [Titanic dataset](https://www.kaggle.com/datasets/brendan45774/test-file) to predict the survival of passengers using [XGBoost][https://es.wikipedia.org/wiki/XGBoost].
Dataset has these relevant fields:
- `Survived`: if passenger survived or not
- `Pclass`: passenger class
- `Sex`: passenger sex
- `Age`: passenger age
- `SibSp`: number of passenger siblings inside the ship
- `Parch`: number of passenger parents inside the ship

This is the workflow that we follow to train and validate the model:

<img src="./diagrams.drawio.png"></img>

## 1. Loading dataset

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb

data = pd.read_csv('./data/titanic.csv')

# debug
print('data columns:', ','.join(data.columns.values))
print('data size: ', data.shape, 'survived:', data['Survived'].value_counts().values)


data columns: PassengerId,Survived,Pclass,Name,Sex,Age,Siblings,Parents,Ticket,Fare,Cabin,Embarked
data size:  (418, 12) survived: [266 152]


## 2. Procesing data
we remove all the columns that are not useful for the model, drop rows with missing values and transform and encode the categorical variables.

In [2]:
data_transformed = data.copy()
data_transformed.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True) # not useful variables
data_transformed.dropna(how='any', inplace=True) # drop rows with missing values

data_transformed['Sex'] = data_transformed['Sex'].map({'male': 0, 'female': 1})
data_transformed = pd.get_dummies(data_transformed, columns=['Embarked', 'Pclass'], drop_first=True) # one-hot encoding for categorical variables

# debug
data_transformed.to_csv('./tmp/titanic_transformed.csv', index=False)
print('data_transformed columns:', ','.join(data_transformed.columns.values))
print('data_transformed size: ', data_transformed.shape)

data_transformed columns: Survived,Sex,Age,Siblings,Parents,Fare,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
data_transformed size:  (331, 10)


## 3. Split dataset into training and test

We need to split the data into training (80%) and testing (20%) sets. 

In [3]:
features = data_transformed.drop('Survived', axis=1)
objective = data_transformed['Survived']

features_train, features_test, objective_train, objective_test = train_test_split(features, objective, test_size=0.20, random_state=42)

# debug
print("Training size", features_train.shape, objective_train.shape)
print("Testing size", features_test.shape, objective_test.shape)


Training size (264, 9) (264,)
Testing size (67, 9) (67,)


# 4. Train the model: XGBoost
Configure and train the model. After training, we will evaluate the model with the test data

In [4]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

model.fit(features_train, objective_train)

train_accuracy = model.score(features_train, objective_train)
test_accuracy = model.score(features_test, objective_test)


# debug
print(f"✅ XGBoost model trained with training accuracy {train_accuracy:.2f}")
print(f"🔎 XGBoost model testing with training accuracy {test_accuracy:.2f}")

✅ XGBoost model trained with training accuracy 1.00
🔎 XGBoost model testing with training accuracy 1.00


# 5. Extract most relevant features
We can also check which features are the most important for the model.
As we expect, the most important feature is sex because all females survived. It's the reason because we have 100% accuracy.

In [5]:
feature_importances = model.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': features_train.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# debug
print("Most important features:")
print(feature_importance_df.head(5))



Most important features:
    Feature  Importance
0       Sex         1.0
1       Age         0.0
2  Siblings         0.0
3   Parents         0.0
4      Fare         0.0


## 6. Make some real predictions
Finally, let's predict the survivability of a passenger

In [12]:

passenger = pd.DataFrame([{
    'Sex': 1,
}])

# Encode data
passenger_encoded = pd.get_dummies(passenger)
passenger_encoded = passenger_encoded.reindex(columns=features_train.columns, fill_value=0) # ensure the columns are the same as the training data

survivability = model.predict_proba(passenger_encoded)
survivability_probability = (survivability[:, 1] * 100).item()

print("Survivability: {:.2f}%".format(survivability_probability))

Survivability: 99.12%
